<a href="https://colab.research.google.com/github/HayateSato/DS_Practice/blob/main/Bean_imageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import hashlib
from collections import Counter # count the occurrences of each class label in image dataset
from PIL import Image #image processing


In [3]:
import os
from google.colab import drive
drive.mount('/content/drive')

train_path = f'/content/drive/MyDrive/Colab Notebooks/Dataset/Bean_leaf_dataset/train'
test_path = f'/content/drive/MyDrive/Colab Notebooks/Dataset/Bean_leaf_dataset/test'
vali_path = f'/content/drive/MyDrive/Colab Notebooks/Dataset/Bean_leaf_dataset/validation'

working_dir = train_path
os.chdir(working_dir)
os.listdir(working_dir)

Mounted at /content/drive


['angular_leaf_spot', 'healthy', 'bean_rust']